In [1]:
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

2023-11-07 15:58:57.840608: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 15:58:57.866168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 15:58:57.866186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 15:58:57.866202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 15:58:57.870941: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 15:58:57.871668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Using TensorFlow backend


In [2]:
adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [3]:
adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=8, max_value=136, step=8)))
    model.add(keras.layers.Dense(units=hp.Int('units2', min_value=4, max_value=32, step=4)))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [5]:
tuner = RandomSearch(
    build_model,
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = 'val_loss',
    max_trials = 2,
    executions_per_trial = 3,
)

In [6]:
tuner.search(adv_X, adv_y, epochs=15, validation_data=(adv_X_test, adv_y_test))

Trial 2 Complete [00h 00m 08s]
val_loss: 0.08559456219275792

Best val_loss So Far: 0.08559456219275792
Total elapsed time: 00h 00m 18s


In [7]:
models = tuner.get_best_models(num_models=2)

In [8]:
best_model = models[0]

In [10]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [11]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  80016     
                                                                 
 dense_1 (Dense)             multiple                  72        
                                                                 
 dense_2 (Dense)             multiple                  9         
                                                                 
Total params: 80097 (312.88 KB)
Trainable params: 80097 (312.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)